## Errors and Exceptions

No matter your skill as a programmer, you will eventually make a coding mistake. Such mistakes come in three basic flavors:

- *Syntax errors*: errors where the code is not valid Python (generally easy to fix)
- *Runtime errors*: errors where syntactically valid code fails to execute, perhaps due to invalid user input (sometimes easy to fix)
- *Semantic errors*: errors in logic; code executes without a problem, but the result is not what you expect (often very difficult to track down and fix)

Here we're going to focus on how to deal cleanly with *runtime errors*. As we'll see, Python handles runtime errors via its *exception handling* framework.

### Runtime Errors

If you've done any coding in Python, you've likely come across runtime errors. They can happen in a lot of ways.

For example, if you try to reference an undefined variable:

In [1]:
print(Q)

NameError: name 'Q' is not defined

Or if you try an operation that's not defined:

In [2]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Or you might be trying to compute a mathematically ill-defined result:

In [3]:
2 / 0

ZeroDivisionError: division by zero

Or maybe you're trying to access a sequence element that doesn't exist:

In [4]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

Note that in each case, Python is kind enough to not only indicate that an error happened, but spits out a *meaningful* exception that includes information about what exactly went wrong, along with the exact line of code where the error happened. Having access to meaningful errors like this is immensely useful when trying to trace the root of problems in your code.

### Catching Exceptions: `try` and `except`

The main tool Python gives you for handling runtime exceptions is the `try`...`except` clause. Its basic structure is this:

In [26]:
try:
    print("This gets executed first")
except:
    print("This gets executed only if there is an error")

This gets executed first


Note that the second block here did not get executed: this is because the first block did not return an error. Let's put a problematic statement in the `try` block and see what happens:

In [6]:
try:
    print("Let's try something:")
    x = 4 / 0 # ZeroDivisionError
except:
    print("Something bad happened!")

Let's try something:
Something bad happened!


Here we see that when the error was raised in the `try` statement (in this case, a `ZeroDivisionError`), the error was caught, and the `except` statement was executed.

One way this is often used is to check user input within a function or another piece of code. For example, we might wish to have a function that catches zero-division and returns some other value, perhaps a suitably large number like 10 x 10^100:

In [7]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [8]:
safe_divide(1, 2)

0.5

In [9]:
safe_divide(2, 0)

1e+100

There is a subtle problem with this code, though: what happens when another type of exception comes up? For example, this is probably not what we intended:

In [10]:
safe_divide(1, '2')

1e+100

Dividing an integer and a string raises a `TypeError`, which our overzealous code caught and assumed was a `ZeroDivisionError`. For this reason, it's nearly always a better idea to catch exceptions *explicitly*:

In [11]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [12]:
safe_divide(1, 0)

1e+100

In [15]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

We're now catching zero-division errors only, and letting all other errors pass through un-modified.

If we wanted to catch a `TypeError`, we could do the following:

In [13]:
def safety_divide(a, b):
    try:
        return a / b
    except TypeError:
        return print("This won't work")

In [14]:
safety_divide(1, '2')

This won't work


### Raising Exceptions: `raise`

We've seen how valuable it is to have informative exceptions when using parts of the Python language. It's equally valuable to make use of informative exceptions within the code you write, so that users of the code (foremost yourself!) can figure out what caused their errors.

The way you raise your own exceptions is with the `raise` statement. For example:

In [16]:
raise RuntimeError("my error message")

RuntimeError: my error message

As an example of where this might be useful, let's return to our `fibonacci` function that we defined previously:

In [17]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

One potential problem here is that the input value could be negative. This will not currently cause any error in our function, but we might want to let the user know that a negative `N` is not supported. Errors stemming from invalid parameter values, by convention, lead to a `ValueError` being raised:

In [18]:
def fibonacci(N):
    if N < 0:
        raise ValueError("this is bad code")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [19]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [20]:
fibonacci(-10)

ValueError: this is bad code

Now the user knows exactly why the input is invalid, and could even use a `try`...`except` block to handle it!

In [22]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    print("bad value: need to do something else")

trying this...
bad value: need to do something else


### Diving Deeper into Exceptions

Let's briefly mention some other concepts we might run into. We won't go into detail on these concepts, but we'll walk through the syntax so you can explore more on your own.

#### Accessing the Error Message

Sometimes in a `try`...`except` statement, you would like to be able to work with the error message itself. This can be done with the `as` keyword:

In [23]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is: ", type(err))
    print("Error message is:", err)

Error class is:  <class 'ZeroDivisionError'>
Error message is: division by zero


With this pattern, you can further customize the exception handling of your function.

#### `try`...`except`...`else`...`finally`

In addition to `try` and `except`, you can use the `else` and `finally` keywords to further tune your code's handling of exceptions. The basic structure is this:

In [24]:
try:
    print("try something here")
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it succeeds
this happens no matter what


The utility of `else` here is clear, but what's the point of `finally`? Well, the `finally` clause really is executed *no matter what*: it's usually used to do some sort of cleanup after an operation completes.